# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time

import pprint

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append(".")
#from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
#    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


# Model preparation 

### Variables

In [2]:
#Inference configs
IMAGE_PATH = '/home/johann/Pictures/images_compl'
DESTINATION_PATH = '/home/johann/Pictures/dest_images'
IMAGE_DIMS = [500, 500]
FIGURE_DIMS = tuple(IMAGE_DIMS.copy())
BATCH_NUM = 20
THRESHOLD = 20

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03'
MODEL_FILE = MODEL_NAME + '.tar.gz'
INFERENCE_GRAPH_PATH = MODEL_NAME + '/frozen_inference_graph.pb'

#Download url
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

VISUALIZE = True

### Model download

In [3]:
if not os.path.isfile(MODEL_FILE):
    print("Downloading model file")
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
else:
    print("Model file already exists")
    
if not os.path.isfile(INFERENCE_GRAPH_PATH):
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        file_name = os.path.basename(file.name)
        if 'frozen_inference_graph.pb' in file_name:
            tar_file.extract(file, os.getcwd())
else:
    print('Model file already extracted')

Model file already exists
Model file already extracted


## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [4]:
CATEGORY_INDEX = {}
f = open("object_detection/mscoco_label_map.pbtxt", "r")
for line in f:
    if 'item' in line:
        name = f.readline().split()[1]
        real_id = int(f.readline().split()[1])
        real_name = f.readline().split()[1]
        CATEGORY_INDEX[real_id] =  {'id': real_id, 'name': real_name}

In [ ]:
print(CATEGORY_INDEX)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic'}, 11: {'id': 11, 'name': 'fire'}, 13: {'id': 13, 'name': 'stop'}, 14: {'id': 14, 'name': 'parking'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name': 'frisbee'}, 35: {'id': 

### Create dataset as tensors

In [ ]:
def create_dataset(path, image_dims, batch_size = 32):
    import pathlib 
    import random 
    print(image_dims)

    
    data_root = pathlib.Path(path)
    image_uris = list(data_root.glob('**/*.jpg'))
    image_uris = [str(image) for image in image_uris]
    
    def preprocess_image(image):

        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize_image_with_pad(image, image_dims[0], image_dims[1])
        #image /= 255  # normalize to [0,1] range

        return image
    
    def load_and_preprocess_image(path):
        
        image = tf.read_file(path)
        
        return preprocess_image(image)
    
    path_ds = tf.data.Dataset.from_tensor_slices(image_uris)
    #AUTOTUNE = tf.data.experimental.AUTOTUNE
    image_ds = path_ds.map(load_and_preprocess_image)
    
    return image_ds

### Create graph and connect tensors

In [ ]:
def build_inference_graph(inference_graph_path, prepend_name):
    """Loads the inference graph and connects it to the input image.
    Args:
    image_tensor: The input image. uint8 tensor, shape=[1, None, None, 3]
    inference_graph_path: Path to the inference graph with embedded weights
    Returns:
    detected_boxes_tensor: Detected boxes. Float tensor,
        shape=[num_detections, 4]
    detected_scores_tensor: Detected scores. Float tensor,
        shape=[num_detections]
    detected_labels_tensor: Detected labels. Int64 tensor,
        shape=[num_detections]
    """
    with tf.gfile.Open(inference_graph_path, 'rb') as graph_def_file:
        graph_content = graph_def_file.read()
    graph_def = tf.GraphDef()
    graph_def.MergeFromString(graph_content)

    tf.import_graph_def(graph_def, name=prepend_name)

    g = tf.get_default_graph()
    #print([n.name for n in tf.get_default_graph().as_graph_def().node])
    
    tensor_dict = {}
    for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes']:
        tensor_name = key + ':0'
        try:
            tensor_dict[key] = g.get_tensor_by_name(prepend_name + '/' + tensor_name)
        except:
            print("Something went horribly wrong when loading graph tensors")
            pass
        
    
    image_tensor = g.get_tensor_by_name(prepend_name + '/' + 'image_tensor:0')
    return {'out': tensor_dict, 'in': image_tensor}, g

# Threaded visualization

In [ ]:
from threading import Thread
from object_detection.utils import visualization_utils as vis_util

def threaded_function(image_np, args_dict, counter, counter2):
    boxes = args_dict['detection_boxes']
    scores = args_dict['detection_scores']
    classes = np.array(args_dict['detection_classes'], np.int16)
    
    for i in range(len(image_np)):
        new_img = np.array(image_np[i], np.uint8)
        vis_util.visualize_boxes_and_labels_on_image_array(
          new_img,
          boxes[i],
          classes[i],
          scores[i],
          CATEGORY_INDEX,
          use_normalized_coordinates=True,
          line_thickness=8)
        plt.figure(figsize=FIGURE_DIMS)
        plt.imsave(DESTINATION_PATH + '/{}_{}_{}.jpg'.format(str(counter), str(counter2), str(i)), new_img)
        plt.close()

In [ ]:
#def threaded_inference(graph):
    #with tf.Session(graph = graph) as sess:
        

# Actual Inference

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    #return [x.name for x in local_device_protos if (x.device_type == 'GPU' or x.device_type == 'CPU')]
    return [x.name for x in local_device_protos if (x.device_type == 'GPU')]

In [ ]:
print(get_available_gpus())

['/device:GPU:0']


In [ ]:
ds = create_dataset(IMAGE_PATH, IMAGE_DIMS)
ds = ds.prefetch(BATCH_NUM*2).batch(BATCH_NUM)
it = ds.make_one_shot_iterator()

# This is needed to display the images.
%matplotlib inline

#config=tf.ConfigProto(log_device_placement=True)

with tf.Session() as sess:
    #tf.logging.info('Reading input from files and connecting to graph')
    #image_tensor = build_input(image_path)
    
    devices = get_available_gpus()
    
    tf.logging.info('Reading graph and building model specified for the different cluster devices')
    
    tf_dict = []
    n = 0
    for d in devices:
        tensors, graph = build_inference_graph(INFERENCE_GRAPH_PATH, 'test')
        tf_dict.append(tensors)
    
    sess.run(tf.global_variables_initializer())
    image_it = it.get_next()
    
    threads = []
    
    i = 1
    j = 1
    try:
        while True:
            print('Step: ' + str(i))
            start = time.time()
            
            output_multi_device = []
            input_feed_dict = {}
            images = []
            
            for idx in tf_dict:
                
                output_multi_device.append(idx['out'])
                #print(idx['out'])
                loc_image = sess.run(image_it)
                print(idx['in'].name)
                input_feed_dict[idx['in']] = loc_image
                images.append(loc_image)
                
            outputs = sess.run(output_multi_device, input_feed_dict)
            #pprint.pprint(outputs)
            
            if VISUALIZE:
                for count in range(len(tf_dict)):
                    thread = Thread(target = threaded_function, args = (images[count], outputs[count], i, count))
                    thread.start()
                    threads.append(thread)
            i += 1
                
        '''
                    image = sess.run(image_it)
                    print('Time required for reading: ' + str(time.time() - start))

                    start = time.time()
                    output_dict = sess.run(tensor_dict, feed_dict = {image_tensor: image})
                    print('Time required for computation: ' + str(time.time() - start))
            
            if VISUALIZE:
                thread = Thread(target = threaded_function, args = (image, output_dict, i))
                thread.start()
                threads.append(thread)
                
            i += 1
        '''
    
    
    except tf.errors.OutOfRangeError:
        tf.logging.info('Finished processing records')
        for t in threads:
            t.join()

[500, 500]
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Reading graph and building model specified for the different cluster devices
Step: 1
test/image_tensor:0
Step: 2
test/image_tensor:0
Step: 3
test/image_tensor:0
Step: 4
test/image_tensor:0
Step: 5
test/image_tensor:0
Step: 6
test/image_tensor:0
Step: 7
test/image_tensor:0
Step: 8
test/image_tensor:0
Step: 9
test/image_tensor:0
Step: 10
test/image_tensor:0
Step: 11
test/image_tensor:0
Step: 12
test/image_tensor:0
Step: 13
test/image_tensor:0
Step: 14
test/image_tensor:0
Step: 15
test/image_tensor:0
Step: 16
test/image_tensor:0
Step: 17
test/image_tensor:0
Step: 18
test/image_tensor:0
Step: 19
test/image_tensor:0
Step: 20
test/image_tensor:0
Step: 21
test/image_tensor:0
Step: 22
test/image_tensor:0
Step: 23
test/image_tensor:0
Step: 24
test/image_tensor:0
Step: 25
test/image_tensor:0
Step: 26
test/image_tensor:0
Step: 27
test/image_tensor:0
Step: 28
test/image_tensor:0
Step: 29
test/image_t

/home/johann/.local/lib/python3.5/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Step: 125
test/image_tensor:0
Step: 126
test/image_tensor:0
Step: 127
test/image_tensor:0
Step: 128
test/image_tensor:0
Step: 129
test/image_tensor:0
Step: 130
test/image_tensor:0
Step: 131
test/image_tensor:0
Step: 132
test/image_tensor:0
Step: 133
test/image_tensor:0
Step: 134
test/image_tensor:0


KeyboardInterrupt: 

In [ ]:
tensor_dict, image_tensor = build_inference_graph(INFERENCE_GRAPH_PATH)


In [ ]:
print(tensor_dict)